# Basics of implementing quantitative trading strategy

*This notebook is an exercise notebook to analyse the Microsoft stocks following the steps mentioned in each cell. Please write the code below the comment or fill the blanks ---------- wherever placed (don't forget to delete the hyphens)*

The OLT covers the following topics:

* Getting started guide
    * Data Collection
    * Time series analysis
    * Exploratory data analysis
    * Visualizations
* Financial Analyses
    * Calculating returns
    * Moving averages
    * Volatility calculation
* Implementing Momentum Trading strategy with Python
* Backtesting the strategy
* Evaluating the strategy
    * Sharpe ratio
    * Max Drawdown
    * Compound Annual Growth Rate


# Data reading and cleansing

In this section, we are going to read the data we extracted from the APIs. We'll explore the data and find the caveats with the format of the data. We'll then make the necessary format changes to the data to make it easy to use while implementing the strategy.

In [5]:
# import the libraries for reading the file from the path


In [ ]:
# read the file from the path `../data/apple_stock_eod_prices.csv'` with 
# index_col = 0, header = 0 and parse_dates = True

msft = --------

# Also print the first  10 rows of the dataframe
print(----------)


# Manipulating Time Series Data

In [ ]:
# inspect the index column
msft.-------


In [ ]:
# inspect and print the columns



In [ ]:
# select and print the  bottom rows


# check the type of the subset rows


In [ ]:
# inspect the data by checking out a sample of 20 rows and print the rows


In [ ]:
# Resample to monthly level using resample() function



In [ ]:
# let's looks at the general statistics of the stock using describe()


Create a diff column inside the 

In [ ]:
# let's look at the difference in the stock price during one day of trading
# subtracting the closing price from the open price and storing it in a new diff column

msft['diff'] = -------------
msft['diff']

In [ ]:
# We should look at the trends and movement of the stock price via some plots
# plotting the closing price using matplotlib which is imported above 
msft['Close'].---------

# show the plot
plt.show()

# Financial Analysis: Calculating Returns

## Returns

In [ ]:
# we'll be using the adjusted closing price column to calculate the percent change 
daily_close = msft[['Adj_Close']]

# using pct_change method to calculate the percent change
daily_pct_c = ----------

# filling the nan values with 0
daily_pct_c.fillna(0, inplace=True)

print(daily_pct_c)

In [ ]:
# Resample `msft` to quarters, take the mean as value per quarter
quarter = ----------

# Calculate the quarterly percentage change
quarter.-------

In [ ]:
# Calculate the daily returns using simple division and shift method
daily_pct_c = -------


In [ ]:
# plot histogram for the above evaluated daily_pct_c with 50 bins



In [ ]:
# Calculate the cumulative daily returns using cumprod()
cum_daily_return = 

# Print `cum_daily_return`
print(cum_daily_return)

# Building the strategy

In [ ]:
# let's add the returns column in the msft  dataframe
msft['returns'] = np.log(msft['Adj_Close'] / msft['Adj_Close'].shift(1))
msft.head()


## Formalizing the momentum strategy

We'll go step by step:
1. First define your two different lookback periods: a short window(30) and a long window(120). Assign these integers to 2 variables. Make sure that the integer that you assign to the short window is shorter than the integer that you assign to the long window variable!

2. Next, make an empty signals DataFrame, but do make sure to copy the index of your msft data so that you can start calculating the daily buy or sell signal for your msft data.

3. Create a column in your empty signals DataFrame that is named signal and initialize it by setting the value for all rows in this column to 0.0.

4. Now that preparatory work is done, it’s time to create the set of short and long simple moving averages over the respective long and short time windows. Make use of the `rolling()` function to start your rolling window calculcations: within the function, specify the window and the min_period, and set the center argument. In practice, this will result in a `rolling()` function to which you have passed either short_window or long_window, 1 as the minimum number of observations in the window that are required to have a value, and `False`, so that the labels are not set at the center of the window. Next, don’t forget to also chain the `mean()` function so that you calculate the rolling mean.

5. Once you have calculated the mean average of the short and long windows, you should create a signal when the short moving average crosses the long moving average, but only for the period greater than the shortest moving average window. In Python, this will result in a condition: **signals['short_mavg'][short_window:] > signals['long_mavg'][short_window:]**. Note that you add the [short_window:] to comply with the condition “only for the period greater than the shortest moving average window”. When the condition is true, the initialized value 0.0 in the signal column will be overwitten with 1.0. A “signal” is created! If the condition is false, the original value of 0.0 will be kept and no signal is generated. You use the NumPy where() function to set up this condition. Much the same like you read just now, the variable to which you assign this result is `signals['signal'][short_window]`, because you only want to create signals for the period greater than the shortest moving average window!

6. Lastly, you take the difference of the signals in order to generate actual trading orders. In other words, in this column of your signals DataFrame, you’ll be able to distinguish between long and short positions, whether you’re buying or selling stock.


**Strategy rules:**

The dual moving average crossover occurs when a short-term average crosses a long-term average. 
This signal is used to identify that momentum is shifting in the direction of the short-term average. A buy signal is generated when the short-term average crosses the long-term average and rises above it, while a sell signal is triggered by a short-term average crossing long-term average and falling below it.



In [ ]:
# Initialize the short and long windows
short_window = ---
long_window = ---

# Initialize the `signals` DataFrame with the `signal` column
signals = pd.DataFrame(index=msft.index)
signals['signal'] = 0.0

# Create short simple moving average over the short window
signals['short_mavg'] = ----

# Create long simple moving average over the long window
signals['long_mavg'] = ----

# Create signals add the logic for checking the crossover of short moving average
signals['signal'][short_window:] =   

# Creating trading orders
signals['positions'] = signals['signal'].diff()
signals

# Plot the above generated signals

In [ ]:
# Initialize the plot figure
fig = plt.figure()

# Add a subplot and label for y-axis
ax1 = fig.add_subplot(111,  ylabel='Price in $')

# Plot the closing price
msft['Close'].plot(ax=ax1, color='r', lw=2.)

# Plot the short and long moving averages
signals[['short_mavg', 'long_mavg']].plot(ax=ax1, lw=2.)

# Plot the buy signals
ax1.plot(signals.loc[signals.positions == 1.0].index, 
         signals.short_mavg[signals.positions == 1.0],
         '^', markersize=10, color='b')
         
# Plot the sell signals
ax1.plot(signals.loc[signals.positions == -1.0].index, 
         signals.short_mavg[signals.positions == -1.0],
         'v', markersize=10, color='k')
         
# Show the plot

# Backtest the strategy

In [ ]:
# Set the initial capital
initial_capital= ----

# Create a DataFrame `positions`
positions = pd.DataFrame(index=signals.index).fillna(0.0)

# Buy a 100 shares
positions['MSFT'] =  ---
  
# Initialize the portfolio with value owned   
portfolio = positions.multiply(msft['Adj_Close'], axis=0)

# Store the difference in shares owned 
pos_diff = positions.diff()

# Add `holdings` to portfolio
portfolio['holdings'] = (positions.multiply(msft['Adj_Close'], axis=0)).sum(axis=1)

# Add `cash` to portfolio
portfolio['cash'] = initial_capital - (pos_diff.multiply(msft['Adj_Close'], axis=0)).sum(axis=1).cumsum()   

# Add `total` to portfolio
portfolio['total'] = portfolio['cash'] + portfolio['holdings']

# Add `returns` to portfolio
portfolio['returns'] = portfolio['total'].pct_change()


In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(111, ylabel='Portfolio value in $')

# Plot the equity curve in dollars
portfolio['total'].plot(ax=ax1, lw=2.)

# Plot the "buy" trades against the equity curve
ax1.plot(portfolio.loc[signals.positions == 1.0].index, 
         portfolio.total[signals.positions == 1.0],
         '^', markersize=10, color='m')

# Plot the "sell" trades against the equity curve
ax1.plot(portfolio.loc[signals.positions == -1.0].index, 
         portfolio.total[signals.positions == -1.0],
         'v', markersize=10, color='k')

# Show the plot
plt.show()

## Calculating the sharpe ratio for moving crossover strategy

In [ ]:
# Sharpe Ratio
# Isolate the returns of your strategy
returns = portfolio['returns']

# annualized Sharpe ratio
sharpe_ratio = np.sqrt(252) * (returns.mean() / returns.std())

# Print the Sharpe ratio
print(sharpe_ratio)

In [ ]:
# Define a trailing 252 trading day window
window = 252

# Calculate the max drawdown in the past window days for each day
rolling_max = ---
daily_drawdown = msft['Adj_Close']/rolling_max - 1.0

# Calculate the minimum (negative) daily drawdown
max_daily_drawdown = ---

# Plot the results
daily_drawdown.plot()
max_daily_drawdown.plot()

# Show the plot
plt.show()

## Compound Annual Growth Rate

In [ ]:
# Get the number of days in `msft`
days = ---

# Calculate the CAGR 
cagr = ----

# Print CAGR
print(cagr)